In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv


In [67]:
import pandas as pd
calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
sales_train_validation = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv")
sample_submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")
sell_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")

In [130]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [69]:
sell_prices.shape

(6841121, 4)

In [70]:

sales_train_validation.shape

(30490, 1919)

In [71]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [132]:
calendar.shape

(1969, 14)

In [133]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [138]:
sales_train_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [291]:
df = sales_train_validation
df = df[df['id']=='FOODS_3_826_WI_3_validation']

In [292]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [325]:
d_cols = [col for col in sales_train_validation.columns if 'd_' in col]

In [349]:
example = df[d_cols].T
example = example.rename(columns={30488:'FOODS_3_826_WI_3'})
example

,FOODS_3_826_WI_3
d_1,0
d_2,0
d_3,0
d_4,0
d_5,0
...,...
d_1909,1
d_1910,0
d_1911,3
d_1912,1


In [350]:
example = example.reset_index().rename(columns={'index': 'd'})

In [351]:
example = example.merge(calendar, how='left', validate='1:1')

In [352]:


example = example.merge(cal, how='left', validate='1:1')

In [353]:
example.head()

,d,FOODS_3_826_WI_3,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,d_2,0,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,d_3,0,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,d_4,0,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0
4,d_5,0,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1


In [354]:
sam = pd.concat([example.FOODS_3_826_WI_3, example.FOODS_3_826_WI_3.shift(-28)], axis=1)

In [355]:
sam

,FOODS_3_826_WI_3,FOODS_3_826_WI_3
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
1908,1,NaN
1909,0,NaN
1910,3,NaN
1911,1,NaN


In [376]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [357]:
example.columns

Index(['d', 'FOODS_3_826_WI_3', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')

In [358]:
example = example.filter(['FOODS_3_826_WI_3','wday','month','year','snap_CA','snap_TX','snap_WI'])

In [359]:
X = example[:-28]

In [360]:
X.columns

Index(['FOODS_3_826_WI_3', 'wday', 'month', 'year', 'snap_CA', 'snap_TX',
       'snap_WI'],
      dtype='object')

In [363]:
y = example.FOODS_3_826_WI_3.shift(-28).dropna()

In [364]:
y

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1880    1.0
1881    0.0
1882    3.0
1883    1.0
1884    3.0
Name: FOODS_3_826_WI_3, Length: 1885, dtype: float64

In [365]:
from sklearn.preprocessing import StandardScaler

In [366]:
scaler = StandardScaler().fit(X)
df_rescaled = scaler.transform(X)

In [367]:
df_rescaled

array([[-0.37438776, -1.4981784 , -1.55441888, ..., -0.70008469,
        -0.70008469, -0.70008469],
       [-0.37438776, -0.99834366, -1.55441888, ..., -0.70008469,
        -0.70008469, -0.70008469],
       [-0.37438776, -0.49850892, -1.55441888, ..., -0.70008469,
        -0.70008469, -0.70008469],
       ...,
       [ 0.66957811,  1.50083004, -0.97814204, ..., -0.70008469,
        -0.70008469, -0.70008469],
       [ 1.71354399, -1.4981784 , -0.97814204, ..., -0.70008469,
        -0.70008469, -0.70008469],
       [ 3.80147573, -0.99834366, -0.97814204, ..., -0.70008469,
        -0.70008469, -0.70008469]])

In [368]:
X

,FOODS_3_826_WI_3,wday,month,year,snap_CA,snap_TX,snap_WI
0,0,1,1,2011,0,0,0
1,0,2,1,2011,0,0,0
2,0,3,1,2011,0,0,0
3,0,4,2,2011,1,1,0
4,0,5,2,2011,1,0,1
...,...,...,...,...,...,...,...
1880,2,5,3,2016,0,0,0
1881,1,6,3,2016,0,0,0
1882,1,7,3,2016,0,0,0
1883,2,1,3,2016,0,0,0


In [369]:

X_train, X_test, y_train, y_test = train_test_split(df_rescaled, y, random_state=42)

In [377]:
rf_model = LogisticRegression()
rf_model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [378]:
rf_model.score(X_test, y_test)

0.8050847457627118